## Homework 7

The risk neutral asset pricing equation for a random payoff $G_{t+1}$ at $t+1$ is given by

$$ p_t = \beta \mathbb{E}_{t} [G_{t+1}]$$

In case of an ex-dividend asset with dividend flow $\{D_t\}$, the above equation becomes

$$ p_t = \beta \mathbb{E}_t[ D_t + p_{t+1} ] $$

For the purposes of the exercise, let the dividends be a invariant function of some state process $\{X_t\}$ that takes values in countable set $S$, that is $D_t = d(X_t)$. Moreover, let the state process be Markov with transition probabilities given by $q(x,y)$.

Note first that equilibrium prices should be a function of current information set, which, in priciple, could include the whole history of endowment process up to $t$. 
However, given our specialization of the divendend process, the only information needed to compute probabilities of future paths of dividends and, hence, the price itself, is the present state value $X_t$. Therefore, we conclude that  price $p_t$ can be represented as a function of that variable only.

So instead of trying to solve for the whole stochastic process $\{p_t\}$ as a function of $\{D_t\}$, our problem reduces to solving for a single pricing function $p: S \rightarrow \mathbb{R}$ that satisfies the following **functional equation**

$$ p(x) = \beta \sum_{y\in S} q(x,y) \big[d(y) + p(y) \big] \quad \forall x \in \tag{1}S$$

#### Theorem 1
If $d\in \mathbb{B}(S)$, then there exists a unique price function $p* \in \mathbb{B}(S)$ that solves (1). Moreover, for any $q\in \mathbb{B}(S)$ we have $T^nq \rightarrow p^*$ where $T:\mathbb{B}(S)\rightarrow \mathbb{B}(S)$ is defined by 

$$ (Tq)(x) \equiv \beta \sum_{y\in S} q(x,y)[d(y) + q(y)] \quad \forall x\in S$$

In [1]:
using QuantEcon

In [2]:
type AssetPrice
    β::Float64
    d::Vector{Float64}
    mc::Vector{QuantEcon.MarkovChain{Float64}}
end

function AssetPrice()
    β  = 0.98
    Π1 = [0.971 0.029 0  ; 0.145 0.778 0.077 ; 0 0.508 0.492 ]
    Π2 = [0.871 0.029 0.1; 0.145 0.778 0.077 ; 0 0.508 0.492 ]
    
    mc1 = MarkovChain(Π1)
    mc2 = MarkovChain(Π2)
    mc  = [mc1,mc2]
    d   = [1.0, 0. , 0.]
    
    AssetPrice(β, d, mc)
end

AssetPrice

In [3]:
function bellman_oper(ap::AssetPrice, q::Vector{Float64}, ind::Int64 = 1)
    β, d, Π = ap.β, ap.d, ap.mc[ind].p
    p = β * Π * (d + q)
    return p 
end

bellman_oper (generic function with 2 methods)

In [4]:
# Initialize an instance of AssetPrice problem
ap = AssetPrice()
bo(q) = bellman_oper(ap,q)

# Use the compute_fixed_point function from QuantEcon to find the fixed point
p0 = [0., 0., 0.]
p₁ = compute_fixed_point(bo, p0, err_tol=1e-5,max_iter=1000, print_skip = 20)

Compute iterate 20 with error 0.547587809459916


3-element Array{Float64,1}:
 40.7593
 35.9604
 34.5715

Note that solution is the same regardless of the initial point chosen

In [5]:
p0 = [0.2, 1., 0.28912]
p₁ = compute_fixed_point(bo, p0, err_tol=1e-5,max_iter=1000, verbose=false)

Compute iterate 40 with error 0.36239836634364053
Compute iterate 60 with error 0.24185476774146863
Compute iterate 80 with error 0.16146188129724948
Compute iterate 100 with error 0.10779317760768237
Compute iterate 120 with error 0.07196358302083894
Compute iterate 140 with error 0.04804346165644091
Compute iterate 160 with error 0.03207419799136346
Compute iterate 180 with error 0.02141299026665422
Compute iterate 200 with error 0.014295483001134812
Compute iterate 220 with error 0.009543778411646997
Compute iterate 240 with error 0.0063715025482835586
Compute iterate 260 with error 0.004253665893287462
Compute iterate 280 with error 0.002839781259538654
Compute iterate 300 with error 0.0018958606069148232
Compute iterate 320 with error 0.0012656916545168428
Compute iterate 340 with error 0.0008449858383414721
Compute iterate 360 with error 0.0005641192816909779
Compute iterate 380 with error 0.000376610529485788
Compute iterate 400 with error 0.0002514281917314065
Compute iterate 4

3-element Array{Float64,1}:
 40.7593
 35.9604
 34.5715

Notice that for this simple example, we could have instead solved directly for $p$. If we assume $S$ finite, as we did in the exercise, (1) turns out to be just a linear system of equations with solution given by

$$ \mathbf{p} = \beta \Pi (\mathbf{d} + \mathbf{p}) \Rightarrow \mathbf{p} = (I - \beta \Pi)^{-1} \beta \Pi \mathbf{d}$$

which gives the same answer (checked below). However, while this approach is only possible in very specific examples, the value function iteration considered first is still valid with no change whatsoever in much more complicated and richer settings. 

In [6]:
β, d, Π = ap.β, ap.d, ap.mc[1].p
pcheck = (eye(3) - β * Π) \ (β * Π * d)

3-element Array{Float64,1}:
 40.7598
 35.9609
 34.572 

Computing the prices according to $\Pi_2$

In [7]:
bo(q) = bellman_oper(ap,q,2)

# Use the compute_fixed_point function from QuantEcon
p0 = [0., 0., 0.]
p₂ = compute_fixed_point(bo, p0, err_tol=1e-5,max_iter=1000, verbose=false)

3-element Array{Float64,1}:
 23.5007
 21.0983
 20.2835

### Exercise 2

Now consider the asset pricing problem with **heterogeneous beliefs**. 
There are two types $i=1,2$ of investors which differ on their beliefs about the markov transition $\Pi_i$. 
Under the assumption that short sales of the asset are not permitted, the equilibrium condition becomes 

$$p(x) = \beta \max_{i\in\{1,2\}} \Big\{ \sum_{y\in S} q_i(x,y) \big[ d(y) + p(y) \big] \Big\} \tag{2}$$

The previous **theorem** asserting existence and uniqueness of an equilibrium still holds. To see that, let $T_1,T_2: \mathbb{B}(S)\rightarrow \mathbb{B}(S)$ denote our previous mappings, where the indexes are associated to the corresponding Markov chain used to form expectations about next period state variable.

In this case, the new equilibrium price becomes the fixed point of the following mapping

$$ (Tq)(x) \equiv \max \Big\{ (T_1q)(x),(T_2q)(x) \Big\} \quad \forall x \in S$$

Since $Tq$ is the upper envelope of two bounded functions, it is itself bounded so all we need to check to guarantee existence and uniqueness of the fixed point is that the operator retains the contraction property of $T_1,T_2$. Let $p,q \in \mathbb{B}(S)$ and $x\in S$. It follows that

$$ 
\begin{align*}
\Big| Tp(x) - Tq(x) \Big| & =  \Big| \max\big\{ (T_1p) (x),(T_2p)(x) \big\} - \max \big\{ (T_1q) (x),(T_2q)(x) \big\} \Big| \\
       &\le \max \bigg\{ \Big| (T_1p) (x) - (T_1q) (x) \Big| , \ \Big| (T_2p) (x) - (T_2q) (x) \Big| \bigg\} \\
       &\le \max \bigg\{ d_{\infty} \big( T_1p, T_1q \big), d_{\infty} \big( T_2p, T_2q \big) \bigg\} \\
       &\le \beta d_{\infty}(p,q)
\end{align*}
$$

where second line follows from tip (proven below), third from definition of $d_{\infty}$ distance and last one from the fact that $T_1,T_2$ are themselves $\beta$-contractions as proved in the first **theorem**. Since $x$ is arbitrary, we have $d_{\infty} (Tp,Tq) \le \beta d_{\infty}(p,q)$ so $T$ is a contraction. 

####  Lemma
Let $(a_i),(b_i) \in \mathbb{R}^{I}$ with $I$ an arbitrary index set. Then the following inequality holds
$$\left| \sup_{i\in I} a_i - \sup_{i\in I}b_i \right| \le \sup_{i\in I} \big|a_i - b_i \big|$$

##### Proof
Note that
$$
\begin{align*}
a_i = ai - b_i + b_i & \le |a_i - b_i| + b_i \\
                     & \le sup_{i\in I} |a_i - b_i| + \sup_{i\in I} b_i  \quad \forall i \in I\\ 
\end{align*}
$$
Since $i$ on the LHS is arbitrary, we have the following inequality
$$ \sup_{i\in I}a_i - \sup_{i\in I} b_i \le sup_{i\in I} |a_i - b_i|  $$
But notice that had we started with $b_i$ we would get the same but with 
$$ \sup_{i\in I}b_i - \sup_{i\in I} a_i \le sup_{i\in I} |a_i - b_i|  $$
which gives the desired inequality.

### Exercise 3

In [8]:
function bellman_oper_het(ap::AssetPrice, q::Vector{Float64})
    β, d, mc = ap.β, ap.d, ap.mc
    Π = [mc[i].p for i in 1:length(mc)]
    
    p = [β * Π[i] * (d + q) for i in 1:length(mc)]
    p = squeeze(maximum( hcat(p...),2 ),2)
    
    return p 
end

bellman_oper_het (generic function with 1 method)

In [9]:
bohet(q) = bellman_oper_het(ap,q)

phb₁ = compute_fixed_point(bohet, p₁, err_tol=1e-5,max_iter=1000, print_skip = 100)

Converged in 1 steps


3-element Array{Float64,1}:
 40.7593
 35.9604
 34.5715

which is *equal* to the price we got before!!! The reason is that in equilibrium with heterogeneous beliefs the agent with more optimistic views of transition probabilities will be the one to hold and, therefore, price the asset in equilibrium. If we take a look at $\Pi_1$ and $\Pi_2$ is easy to check that agent 1 is always more optmistic than agent 2, so we get the same prices as before

In [10]:
q = phb₁
β, d, mc = ap.β, ap.d, ap.mc
Π = [mc[i].p for i in 1:length(mc)]

p = [β * Π[i] * (d + q) for i in 1:length(mc)]
hcat(p...)

3x2 Array{Float64,2}:
 40.7593  40.0549
 35.9604  35.9604
 34.5715  34.5715

If we replace agent 2's probability assessment to 

In [11]:
Π2alt = [0.871 0.029 0.1; 0.212 0.778 0.01 ; 0 0.508 0.492 ]

mc2 = copy(mc)
mc2[2] = MarkovChain(Π2alt)

ap2 = AssetPrice(ap.β, ap.d, mc2)

AssetPrice(0.98,[1.0,0.0,0.0],QuantEcon.MarkovChain{Float64,TM<:AbstractArray{T,2},TV<:AbstractArray{T,1}}[Discrete Markov Chain
stochastic matrix of type Array{Float64,2}:
3x3 Array{Float64,2}:
 0.971  0.029  0.0  
 0.145  0.778  0.077
 0.0    0.508  0.492
,Discrete Markov Chain
stochastic matrix of type Array{Float64,2}:
3x3 Array{Float64,2}:
 0.871  0.029  0.1  
 0.212  0.778  0.01 
 0.0    0.508  0.492
])

we get that the asset will change hands in equilibrium

In [12]:
bohet(q) = bellman_oper_het(ap2,q)

phb₂ = compute_fixed_point(bohet, phb₁, err_tol=1e-5,max_iter=1000, verbose=false)

Converged in 429 steps


3-element Array{Float64,1}:
 43.3678
 40.4046
 38.8441

and find that the equilibrium price is actually strictly larger than each agent's expected discounted value of future dividends. 
*In equilibrium, each agent attributes a higher value to the asset because he knows he can sell it to the other type on states where his valuation is below the actual price*

In [13]:
β, d, mc = ap2.β, ap2.d, ap2.mc
Π = [mc[i].p for i in 1:length(mc)]

p = [β * Π[i] * (d + phb₂) for i in 1:length(mc)]
hcat(p...)

3x2 Array{Float64,2}:
 43.3678  42.8265
 40.0419  40.4046
 38.8441  38.8441

## SUMMARY

state $\ $ |$\ $ normal growth  $\ $|$\ $ mild recession $\ $|$\ $ severe recession $\ $
:-------------:|:-------------:|:--------------:|:----------------:
               |               |                |   
$p_1$          | 40.7593       | 35.9604        | 34.5715  
$p_2$          | 23.5007       | 21.0983        | 20.2835  
$p^{hb}_1$     | 40.7593       | 35.9604        | 34.5715  
$p^{hb}_2$     | 43.3678       | 40.4046        | 38.8441